<a href="https://colab.research.google.com/github/michaelbarkasi/Intro_Cog_Sci_Examples/blob/main/PNP_200_Sample_PSS_River_Crossing_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Physical Symbol System

CC Michael Barkasi 2024

[Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/) //
[BSD (3-Clause) License](https://opensource.org/licenses/BSD-3-Clause)

## Intro

What is this? Sample code and walkthrough of a Physical Symbol System -- a virtual one, of course. My Python is rusty, so I asked ChatGPT to write me some of the first few (simple) chunks of Python code used below. I checked and cleaned them. The second half of the code is more me writing from scratch.

Note: All code chunks must be run in order. Rerunning chunks in different orders might change results, because variables get be redefined with new input.

## Physical Symbol Systems

In 1975, Herbert Simon and Allen Newell proposed the *physical symbol system hypothesis*: A physical symbol system has the necessary and sufficient means for general intelligent action. They defined a physical symbol system as a physical device or other physical system:
1. with physical patterns (the symbols)
2. that can be combined to form complex symbol structures
3. such that the system contains processes or mechanisms for manipulating those complex symbol structures
4. and such that the processes for generating and transforming those complex symbol structures can themselves be represented by complex symbol structures within the system.

Simon and Newell don't have a precise definition of "intelligent action", but instead think of it on the model of problem solving via a search space.

## River Crossing Problem

This CoLab will simulate a physical symbol system (PSS), using Python, which demonstrates "intelligent action" by solving one version of the *River Crossing Problem*. The problem goes: Kirk, Spock, and McCoy (three officers) and three “red shirts” are on one side of a river with a small boat that holds at most two people. Gorn are lurking on both river banks, and if one or two of the red shirts ever finds themselves outnumbered by officers (“odd ones out”), the Gorn attack. How can all six get to the other side of the river without inviting a Gorn attack? (This version of the problem is inspired by chapter 4 of [Bermúdez's textbook, _Cognitive Science_](https://www.cambridge.org/core/resources/cognitive-science-4/).)

We can think of the problem as involving an initial state, a solution state, and a set of constraints on how to get from one to the other. The initial state is all six people on one bank, the solution state is all six on the opposite bank, the constraints are that only at most two people can fit in the boat, someone always has to pilot the boat across the river (of course), and if there are red shirts on a bank, they can’t be outnumbered by officers.

For those interested, read about the original / classic River Crossing Problem here: [Wikipedia](https://en.wikipedia.org/wiki/Wolf,_goat_and_cabbage_problem)

## Basic Parameters

**Symbols:** o, r, b, /

**Complex Symbol Structures:** Any string with exactly 8 characters that has 0-3 o followed by 0-3 r, followed possibly by 0-1 b, followed by /, followed by 0-3 o, followed by 0-4 r, followed by 0-1 b.

**Manipulation Processes:** Move at least one and at most two other characters and the b to the opposite side of the /, such that if there’s an r on a side of the /, the number of o never exceeds the number of r on that side.

**Initial State:** ooorrrb/

**Solution State:** /ooorrrb

In [ ]:
# @title CoLab Reset
# Want to reset loaded code?
%reset -f

In [ ]:
# @title Initiate State Variables

# Initialize the State variable as an empty 1D array of length 8
State = [''] * 8

# Initialize the Initial State variable with the specified characters
Initial_State = list("ooorrrb/")

# Initialize the Solution State variable with the specified characters
Solution_State = list("/ooorrrb")

'''
  Explanation: The "/" represents the river. "o" represents an officer, "r" a red shirt, and "b" the boat.
    If an "o", "r", or "b" is on the left side of the "/", that represents it as being on the left side of the river.
    If an "o", "r", or "b" is on the right side of the "/", that represents it as being on the right side of the river.
'''

# Printing the initialized variables for verification
print("State:", State)
print("Initial State:", Initial_State)
print("Solution State:", Solution_State)

State: ['', '', '', '', '', '', '', '']
Initial State: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/']
Solution State: ['/', 'o', 'o', 'o', 'r', 'r', 'r', 'b']


In [ ]:
# @title Manipulation Functions: Boat Side Check

'''
 Next, we need to define functions which simulate the manipulation processes of our PSS,
  while respecting the constraints of the problem.
'''

# First we need a function to check which side of the river the boat is on
def boat_on_left_of_river(State):
    try:
        # A state variable is a string of 8 characters.
        # Begin by finding the place (the "index") of of the river character (/) and the boat character (b).
        slash_index = State.index('/')
        b_index = State.index('b')

        # If the boat character index is less than the river character index,
        #   then that represents the boat as being on the left side of the river.
        if b_index < slash_index:
            return True # yes, boat is on the left side of river
        else:
            return False # no, boat is not on the left side of river
    except ValueError:
        # Handle the case where either 'b' or '/' is not found in State
        return False

# Example usage:
State = list("ooorr/orb")  # Replace with your State variable
test_result = boat_on_left_of_river(State)
print(f'In State {State}:')
if test_result:
    print("The boat is on the left side of the river.")
else:
    print("The boat is on the right side of the river.")

State = list("ooorrrb/o")  # Replace with your State variable
test_result = boat_on_left_of_river(State)
print(f'\nIn State {State}:')
if test_result:
    print("The boat is on the left side of the river.")
else:
    print("The boat is on the right side of the river.")

In State ['o', 'o', 'o', 'r', 'r', '/', 'o', 'r', 'b']:
The boat is on the right side of the river.

In State ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/', 'o']:
The boat is on the left side of the river.


In [ ]:
# @title Manipulation Functions: Cross River

'''
 We'll also need a function which simulates a river crossing.
 Remember: If the boat is on the left side, it can carry at most two people (but must have at least one, a pilot).
           If the boat is on the right side (and we're not in the solution state), someone will have to pilot it back to the left side
            to pick up another person. Of course, two can go back as well (this is important, to avoid leaving an odd redshirt out).
'''

# We need to important the random function
import random

# Helper function
def identity_function2(x,y):
    return x
def identity_function3(x,y,z):
    return x

def cross_river(State,
                heuristics_func_left=identity_function2,
                heuristics_func_right=identity_function3):

    print(f"Current state: {State}.")
    # Check if 'b' is on the left or right of '/'
    if boat_on_left_of_river(State):
        print("Boat is on left.")
        left_side = State[:State.index('/')] # array indexes of all characters to the left of slash
        right_side = State[State.index('/')+1:] # ... and to the right

        # Randomly select 1 or 2 characters from the left side
        num_to_move = random.randint(1, 2)
        left_side.remove('b') # 'b' is on the left side, but isn't a character (person), so don't want it randomly selected.
        if len(left_side) == 1:
          num_to_move = 1
        characters_to_move = random.sample(left_side, num_to_move)
        # We won't be using heuristics for now, but, defining this here for use later
        characters_to_move = heuristics_func_left(characters_to_move,left_side)
        print(f"Crossing river with characters: {', '.join(characters_to_move)}")

        # Move the selected characters to the right side
        right_side.extend(characters_to_move)
        for char in characters_to_move:
            left_side.remove(char)

        # Update the State
        State = left_side + ['/'] + right_side
        State.append('b') # Have to put 'b' back, so it gets moved as well.
    else:
        print("Boat is on right.")
        left_side = State[:State.index('/')]
        right_side = State[State.index('/')+1:]

        # Randomly select 1 or 2 characters from the right side
        num_to_move = random.randint(1, 2)
        right_side.remove('b')
        if len(right_side) == 1:
          num_to_move = 1
        characters_to_move = random.sample(right_side, num_to_move)
        # We won't be using heuristics for now, but, defining this here for use later
        characters_to_move = heuristics_func_right(characters_to_move,left_side,right_side)
        print(f"Crossing river with characters: {', '.join(characters_to_move)}")

        # Move the selected characters to the left side
        left_side.extend(characters_to_move)
        for char in characters_to_move:
            right_side.remove(char)

        # Update the State
        State = left_side + ['b'] + ['/'] + right_side

    return State

# Example usage:
State = list("ooorrrb/")
new_state = cross_river(State)
print(f"New State after crossing: {new_state}\n")

State = list("or/oorrb")
new_state = cross_river(State)
print(f"New State after crossing: {new_state}\n")

State = list("oob/orrr")
new_state = cross_river(State)
print(f"New State after crossing: {new_state}\n")

State = list("ro/oorrb")
new_state = cross_river(State)
print(f"New State after crossing: {new_state}\n")

Current state: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/'].
Boat is on left.
Crossing river with characters: r, o
New State after crossing: ['o', 'o', 'r', 'r', '/', 'r', 'o', 'b']

Current state: ['o', 'r', '/', 'o', 'o', 'r', 'r', 'b'].
Boat is on right.
Crossing river with characters: r, o
New State after crossing: ['o', 'r', 'r', 'o', 'b', '/', 'o', 'r']

Current state: ['o', 'o', 'b', '/', 'o', 'r', 'r', 'r'].
Boat is on left.
Crossing river with characters: o
New State after crossing: ['o', '/', 'o', 'r', 'r', 'r', 'o', 'b']

Current state: ['r', 'o', '/', 'o', 'o', 'r', 'r', 'b'].
Boat is on right.
Crossing river with characters: r, o
New State after crossing: ['r', 'o', 'r', 'o', 'b', '/', 'o', 'r']



In [ ]:
# @title Manipulation Functions: Red Shirt Check

'''
 The final constraint of the problem is that if there are red shirts on a bank, they can’t be outnumbered by officers.
 So we need a check function which looks to see if a given output state has more 'o' than 'r' on one side.
'''

def odd_redshirt_out(State):

    # Begin by grabbing lists of the characters on each side of the river
    left_side = State[:State.index('/')]
    right_side = State[State.index('/')+1:]

    # Check if 'r' is present on either side
    left_has_r = 'r' in left_side
    right_has_r = 'r' in right_side

    # Count 'o' and 'r' on each side
    left_o_count = left_side.count('o')
    left_r_count = left_side.count('r')

    right_o_count = right_side.count('o')
    right_r_count = right_side.count('r')

    # Check if there is 'r' and more 'o' than 'r' on either sides
    left_condition = left_has_r and left_o_count > left_r_count
    right_condition = right_has_r and right_o_count > right_r_count

    return left_condition or right_condition # This will be a Boolean value (TRUE or FALSE) telling us whether a redshirt is outnumbred by officers.

# Example usage:
State1 = list("ooorrrb/")  # Replace with your State variable
State2 = list("rooo/rrb")
State3 = list("oorrrb/o")
State4 = list("ooorrb/r")

print(f'Any odd redshirts out State1? {odd_redshirt_out(State1)}.\n')
print(f'Any odd redshirts out State2? {odd_redshirt_out(State2)}.\n')
print(f'Any odd redshirts out State3? {odd_redshirt_out(State3)}.\n')
print(f'Any odd redshirts out State4? {odd_redshirt_out(State4)}.\n')

Any odd redshirts out State1? False.

Any odd redshirts out State2? True.

Any odd redshirts out State3? False.

Any odd redshirts out State4? True.



In [ ]:
# @title Random Brute-Force Solution Finder

# To keep output tidy
from IPython.display import clear_output
keep_tidy = True

'''
 Finally, we're ready to put these functions together in a way which solves the problem!

 Because we're not being systematic, sometimes dead ends happen: e.g., we end up in a state like ro/oorrb in which
  all possible crossings leave an odd redshirt out. If we do nothing, the PSS will get stuck in an infinite loop at these
  points. So, we need it to check that it's not in a loop, and, if in a loop, to halt and report a failed search.
'''

def River_crossing_solution_finder(Initial_State_,
                                   Final_State_,
                                   cross_func,
                                   heuristics_func_left=identity_function2,
                                   heuristics_func_right=identity_function3,
                                   max_crossings=50):

  # Initiate variables to keep track of the number of crossings made,
  #  and also the number of failed crossings (crossings which leave a redshirt as the odd one out).
  failed_crossings = 0
  crossings = 0

  # Initiate a variable to keep track of the current state and initiate it in the problem's Initial State.
  State = Initial_State

  print(f"\nBeginning in Initial State: {State}\n")

  # Continually loop until a return is triggered
  while True:

    # Step 1: Perform a random river crossing
    #   ... store in a temporary variable, until we check that the crossing didn't fail.
    State_new = cross_func(State, heuristics_func_left, heuristics_func_right)

    # Step 2: Check if this crossing left any odd redshirts out (i.e., if it failed)
    #   ... if not, update State
    if not odd_redshirt_out(State_new):
      State = State_new
      crossings += 1
      print(f"Crossing {crossings}, New State after crossing: {State}\n")
    else:
      failed_crossings += 1
      print(f"Aborting attempt! Would be failed crossing / leave red shirt odd-one-out, Gorn attack!")

    # Step 3: Check if we're in the solution state
    '''
      We're cheating a little here; instead of comparing the whole "complex symbol" State to Solution_State,
      we're just comparing their first character. This is because, really, the "solution" is just to have the slash
      as the first character (i.e., to get all characters to the right of the slash).
    '''
    if State[0] == Solution_State[0]:
      print(f"Solution State Found!\n")
      print(f"Number of crossings used (length of solution path): {crossings}\n")
      return(True)

    # Check if we're stuck. (If more than 10 failed crossings, abort and start over.)
    if failed_crossings > 10 or crossings > max_crossings:
      return(False)

found_solution = False
Try = 0

# Loop the solution_finder until a solution is found.
while not found_solution:

  # Keep track of how many times we have to run the solution finder until it succeeds.
  Try += 1

  # Run solution_finder and check if it finds a solution.
  if River_crossing_solution_finder(Initial_State,Solution_State,cross_river):
    found_solution = True
    print(f'Total number of paths searched to find solution: {Try}\n')
  else:
    print("\nTrapped by Gorn! Failed search path.")
    if keep_tidy:
      clear_output()


Beginning in Initial State: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/']

Current state: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/'].
Boat is on left.
Crossing river with characters: o, r
Crossing 1, New State after crossing: ['o', 'o', 'r', 'r', '/', 'o', 'r', 'b']

Current state: ['o', 'o', 'r', 'r', '/', 'o', 'r', 'b'].
Boat is on right.
Crossing river with characters: r
Crossing 2, New State after crossing: ['o', 'o', 'r', 'r', 'r', 'b', '/', 'o']

Current state: ['o', 'o', 'r', 'r', 'r', 'b', '/', 'o'].
Boat is on left.
Crossing river with characters: o
Crossing 3, New State after crossing: ['o', 'r', 'r', 'r', '/', 'o', 'o', 'b']

Current state: ['o', 'r', 'r', 'r', '/', 'o', 'o', 'b'].
Boat is on right.
Crossing river with characters: o
Crossing 4, New State after crossing: ['o', 'r', 'r', 'r', 'o', 'b', '/', 'o']

Current state: ['o', 'r', 'r', 'r', 'o', 'b', '/', 'o'].
Boat is on left.
Crossing river with characters: o, o
Crossing 5, New State after crossing: ['r', 'r', 'r', '/', 'o

## Conclusion

There you go! Because the algorithm is randomly selecting characters to move at each "crossing", each time it's run it will take a different number of path searches to find a solution path, and it will find different solution paths of different lengths.

# Exercise: Improve Algorithm with Heuristics

The above solution used a brute-force method to solve the problem. In essence, it randomly searches all possible paths in the "crossing space" until it finds a crossing which solves the problem. Because it's random, the solutions it finds are normally longer (more crossings) than a person could devise with reason, and it often takes many tries until it finds a solution, because its random selection often leads it into dead ends in which a red shirt is always left outnumbered by officers. (For example, I solved the problem by hand in 12 crossings, as outlined in the lecture slides.)

What we want to do now is program two heuristic rules which will help speed up our search and shorten the solution found to 15 or less crossings.

To ensure we have 15 or fewer crossings, we will simply set the solution_finder so that it exits as a failed attempt if there are more than 15 crossings.

More importantly, we will now set a heuristic rule, one for how to handle crossing left-to-right, and one for how to handle crossing right-to-left. In English, the heuristics we'll use are:

**Left-to-Right:** If a passenger can be taken, then take them. (Never cross with just a boat pilot, if there are more than two people on the left.)

**Right-to-Left:** If all three red shirts are on the right side of the river and there is also an officer on this side of the river who can pilot the boat back to the left side, then the officer should pilot the boat back alone.

Exercise instructions: Fill in the code below to define precise functions for these heuristic rules. Then, run the code to see what the algorithm finds.

In [ ]:

# Define heuristic functions for crossing left-to-right and right-to-left

maximum_crossings = 15

def force_passenger(original_list,people_left):
  if FILL-IN-CONDITION-HERE:
    return FILL-IN-WHAT-TO-RETURN
  else:
    return original_list

def keep_redshirts_right(original_list,people_left,people_right):
  if FILL-IN-CONDITION-HERE:
    return FILL-IN-WHAT-TO-RETURN
  else:
    return original_list

# Now rerun the solution finder using these heuristics
found_solution = False
Try = 0

# Loop the solution_finder until a solution is found.
while not found_solution:

  # Keep track of how many times we have to run the solution finder until it succeeds.
  Try += 1

  # Run solution_finder and check if it finds a solution.
  if River_crossing_solution_finder(Initial_State,Solution_State,cross_river,
                                    heuristics_func_left=force_passenger,
                                    heuristics_func_right=keep_redshirts_right,
                                    max_crossings=maximum_crossings):
    found_solution = True
    print(f'Total number of paths searched to find solution: {Try}\n')
  else:
    print("\nTrapped by Gorn! Failed search path.")
    if keep_tidy:
      clear_output()


Beginning in Initial State: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/']

Current state: ['o', 'o', 'o', 'r', 'r', 'r', 'b', '/'].
Boat is on left.
Crossing river with characters: o, o
Crossing 1, New State after crossing: ['o', 'r', 'r', 'r', '/', 'o', 'o', 'b']

Current state: ['o', 'r', 'r', 'r', '/', 'o', 'o', 'b'].
Boat is on right.
Crossing river with characters: o, o
Crossing 2, New State after crossing: ['o', 'r', 'r', 'r', 'o', 'o', 'b', '/']

Current state: ['o', 'r', 'r', 'r', 'o', 'o', 'b', '/'].
Boat is on left.
Crossing river with characters: r, r
Aborting attempt! Would be failed crossing / leave red shirt odd-one-out, Gorn attack!
Current state: ['o', 'r', 'r', 'r', 'o', 'o', 'b', '/'].
Boat is on left.
Crossing river with characters: r, r
Aborting attempt! Would be failed crossing / leave red shirt odd-one-out, Gorn attack!
Current state: ['o', 'r', 'r', 'r', 'o', 'o', 'b', '/'].
Boat is on left.
Crossing river with characters: o, o
Crossing 3, New State after crossing: [

# Conclusion

You should see how the new algoritm, using heuristics, is much faster and returns a shorter crossing path (always under 15 crossings long) than the original, brute-force random search algorithm!

# Solution Code

See below for a possible solution (two heuristic functions which will work).

In [ ]:
# @title Solution

def force_passenger(original_list,people_left):
  if len(people_left) > 1:
    return random.sample(people_left, 2)
  else:
    return original_list

def keep_redshirts_right(original_list,people_left,people_right):
  if 'r' not in people_left and 'o' in people_right:
    return list("o")
  else:
    return(original_list)